In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# Helium 

## Initialize matrix parser

In [ ]:
from read_data import MatrixElementParser

Z = 2
matr = MatrixElementParser(Z = Z)

## Find reference energy

In [ ]:
Eref = 0
for i in range(2):
    for j in range(2):
        Eref += 0.5*matr[i,j,i,j]
    Eref += onebody(i,i)
Eref

## Define hamiltonian up to singles-singles

In [ ]:
def onebody(p,q):
    n = p//2 + 1
    if n != q//2 + 1:
        return 0
    else:
        return -Z**2/(2*n**2)
    
n_states = 5

core = np.arange(F)
n_virtual = 4
virtual = np.arange(n_virtual)

refs = [0,1]
nref = len(refs) # this code needs a bit of rework if the ground state is degenerate

singles = [] # [(0,2),(1,3),(0,4),(1,5)]

for i in core:
    for a in virtual:
        if (i%2) == (a%2):
            singles.append((i,a))
            
nsingles = len(singles)

ref_ref = np.zeros((1,1))
ref_sing = np.zeros((nsingles,1))
sing_sing = np.zeros((nsingles,nsingles))

ref_ref[0,0] = Eref
for ind1,(i,a) in enumerate(singles):
    for ind2,(j,b) in enumerate(singles):
        E = matr[a,j,i,b] 
        E += onebody(a,b) if i==j else 0
        E -= onebody(i,j) if a==b else 0
        E += Eref if (i==j and a==b) else 0
        sing_sing[ind1,ind2] = E
    
    ref_sing[ind1] = sum(matr[i,j,a,j] for j in core) + onebody(i,a)

                    
H = np.block([[ref_ref, ref_sing.T],
              [ref_sing, sing_sing]])

e,v = np.linalg.eig(H)
print('eigenvalues')
print(e)